<a href="https://colab.research.google.com/github/DenisKai7/id_scanner/blob/model-train/id_scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-ind

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-ind
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 537 kB of archives.
After this operation, 1,138 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ind all 1:4.00~git30-7274cfa-1.1 [537 kB]
Fetched 537 kB in 0s (2,095 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend:

In [3]:
!pip install pytesseract

In [4]:
!pip install opencv-python

In [5]:
import cv2
import pytesseract
from PIL import Image
import os
import re
import numpy as np

In [13]:
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata'

In [15]:
print("\nIsi folder tessdata:")
!ls /usr/share/tesseract-ocr/4.00/tessdata/


Isi folder tessdata:
configs		 ind.traineddata  pdf.ttf
eng.traineddata  osd.traineddata  tessconfigs


In [14]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [7]:
base_drive_path = '/content/drive/MyDrive/ocr_ktp/train'

In [8]:
def correct_rotation(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Tidak dapat membaca gambar dari {image_path}")
        return None

    # ubah gambar ke grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    try:
        osd = pytesseract.image_to_osd(gray)
        rotation_angle = 0
        for line in osd.split('\n'):
            if 'Rotate:' in line:
                rotation_angle = int(line.split('Rotate: ')[1])
                break

        if rotation_angle == 0:
            return img

        (h, w) = img.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, -rotation_angle, 1.0)
        rotated_img = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        return rotated_img
    except pytesseract.TesseractError as e:
        print(f"Peringatan: Gagal mendeteksi orientasi dengan Tesseract OSD: {e}. Mencoba OCR langsung.")

        return img

In [9]:
def extract_ktp_data(image_path):
    print(f"\nMemproses KTP: {image_path}")
    # Koreksi rotasi gambar terlebih dahulu
    processed_img = correct_rotation(image_path)
    if processed_img is None:
        return None

    pil_img = Image.fromarray(cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB))

    text = pytesseract.image_to_string(pil_img, lang='ind', config='--psm 6 --oem 3')

    data_ktp = {
        "NIK": "",
        "Nama": "",
        "Tempat/Tgl Lahir": "",
        "Jenis Kelamin": "",
        "Gol. Darah": "",
        "Alamat": {
            "Jalan": "",
            "RT/RW": "",
            "Kel/Desa": "",
            "Kecamatan": "",
            "Agama": "",
            "Status Perkawinan": "",
            "Pekerjaan": "",
            "Kewarganegaraan": "",
            "Berlaku Hingga": ""
        }
    }

    # Regex untuk ekstrak data
    # NIK: Pola 16 digit angka
    nik_match = re.search(r'\b\d{16}\b', text)
    if nik_match:
        data_ktp["NIK"] = nik_match.group(0)

    # Nama: Cari setelah kata "Nama" atau "Nama Lengkap"
    name_match = re.search(r'(?:Nama|Nama Lengkap)\s*:\s*(.+)', text, re.IGNORECASE)
    if name_match:
        data_ktp["Nama"] = name_match.group(1).strip()
    else:
        name_match = re.search(r'Nama\s*([A-Z\s]+)', text)
        if name_match:
            data_ktp["Nama"] = name_match.group(1).strip()


    # Tempat/Tgl Lahir: Cari setelah "Tempat/Tgl Lahir" atau "Tempat, Tgl Lahir"
    dob_match = re.search(r'(?:Tempat/Tgl Lahir|Tempat, Tgl Lahir)\s*:\s*([A-Za-z\s]+),\s*(\d{2}-\d{2}-\d{4})', text, re.IGNORECASE)
    if dob_match:
        data_ktp["Tempat/Tgl Lahir"] = f"{dob_match.group(1).strip()}, {dob_match.group(2).strip()}"

    # Jenis Kelamin: Cari setelah "Jenis Kelamin"
    gender_match = re.search(r'Jenis Kelamin\s*:\s*(LAKI-LAKI|PEREMPUAN)\s*(?:Gol\. Darah\s*:\s*([A|B|AB|O|\-]+))?', text, re.IGNORECASE)
    if gender_match:
        data_ktp["Jenis Kelamin"] = gender_match.group(1).strip()
        if gender_match.group(2):
            data_ktp["Gol. Darah"] = gender_match.group(2).strip()

    # Gol. Darah: Jika tidak terdeteksi bersama Jenis Kelamin
    if not data_ktp["Gol. Darah"]:
        blood_match = re.search(r'Gol\. Darah\s*:\s*([A|B|AB|O|\-]+)', text, re.IGNORECASE)
        if blood_match:
            data_ktp["Gol. Darah"] = blood_match.group(1).strip()

    # Alamat
    address_start_idx = text.lower().find("alamat")
    if address_start_idx != -1:
        address_text = text[address_start_idx:]
        # Jalan
        jalan_match = re.search(r'Jalan\s*:\s*(.+)', address_text, re.IGNORECASE)
        if jalan_match:
            data_ktp["Alamat"]["Jalan"] = jalan_match.group(1).strip()

        # RT/RW
        rtrw_match = re.search(r'RT/RW\s*:\s*(\d{3}/\d{3})', address_text, re.IGNORECASE)
        if rtrw_match:
            data_ktp["Alamat"]["RT/RW"] = rtrw_match.group(1).strip()

        # Kel/Desa
        keldesa_match = re.search(r'(?:Kel/Desa|Kelurahan/Desa)\s*:\s*(.+)', address_text, re.IGNORECASE)
        if keldesa_match:
            data_ktp["Alamat"]["Kel/Desa"] = keldesa_match.group(1).strip()

        # Kecamatan
        kec_match = re.search(r'Kecamatan\s*:\s*(.+)', address_text, re.IGNORECASE)
        if kec_match:
            data_ktp["Alamat"]["Kecamatan"] = kec_match.group(1).strip()

    # Agama
    agama_match = re.search(r'Agama\s*:\s*(ISLAM|KRISTEN|KATOLIK|HINDU|BUDHA|KONGHUCU)', text, re.IGNORECASE)
    if agama_match:
        data_ktp["Alamat"]["Agama"] = agama_match.group(1).strip()

    # Status Perkawinan
    status_match = re.search(r'Status Perkawinan\s*:\s*(BELUM KAWIN|KAWIN|CERAI HIDUP|CERAI MATI)', text, re.IGNORECASE)
    if status_match:
        data_ktp["Alamat"]["Status Perkawinan"] = status_match.group(1).strip()

    # Pekerjaan
    pekerjaan_match = re.search(r'Pekerjaan\s*:\s*(.+)', text, re.IGNORECASE)
    if pekerjaan_match:
        data_ktp["Alamat"]["Pekerjaan"] = pekerjaan_match.group(1).strip()

    # Kewarganegaraan
    warga_match = re.search(r'Kewarganegaraan\s*:\s*(WNI|WNA)', text, re.IGNORECASE)
    if warga_match:
        data_ktp["Alamat"]["Kewarganegaraan"] = warga_match.group(1).strip()

    # Berlaku Hingga
    berlaku_match = re.search(r'Berlaku Hingga\s*:\s*(\d{2}-\d{2}-\d{4}|SEUMUR HIDUP)', text, re.IGNORECASE)
    if berlaku_match:
        data_ktp["Alamat"]["Berlaku Hingga"] = berlaku_match.group(1).strip()

    return data_ktp

In [ ]:
all_ktp_data = []

for angle_folder in ['0', '90', '180', '270']:
    folder_path = os.path.join(base_drive_path, angle_folder)
    print(f"Mengakses folder: {folder_path}")

    if not os.path.exists(folder_path):
        print(f"Peringatan: Folder '{folder_path}' tidak ditemukan. Pastikan path sudah benar.")
        continue

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            extracted_data = extract_ktp_data(image_path)
            if extracted_data:
                all_ktp_data.append(extracted_data)


Mengakses folder: /content/drive/MyDrive/ocr_ktp/train/0

Memproses KTP: /content/drive/MyDrive/ocr_ktp/train/0/IMG-20230726-WA0052_jpg.rf.d5947bfeea9669c5e0de1e48d0d01f86_ktp_0.jpg

Memproses KTP: /content/drive/MyDrive/ocr_ktp/train/0/IMG-20230726-WA0052_jpg.rf.b18f09ca41b16a271c288f750e9f0fb3_ktp_0.jpg

Memproses KTP: /content/drive/MyDrive/ocr_ktp/train/0/IMG-20230729-WA0024_jpg.rf.b9d531154b08febf0ae03d57fcb4cfde_ktp_0.jpg

Memproses KTP: /content/drive/MyDrive/ocr_ktp/train/0/IMG-20230729-WA0013_jpg.rf.91793a02446ad1438bb5ceba23f8115d_ktp_0.jpg

Memproses KTP: /content/drive/MyDrive/ocr_ktp/train/0/IMG-20230729-WA0022_jpg.rf.041911ebe01d51405546c15f72c70217_ktp_0.jpg

Memproses KTP: /content/drive/MyDrive/ocr_ktp/train/0/IMG-20230729-WA0013_jpg.rf.c50fbbd70fc782c39e9d583c7be5faf8_ktp_0.jpg

Memproses KTP: /content/drive/MyDrive/ocr_ktp/train/0/IMG-20230729-WA0013_jpg.rf.0017a64618c047a3d597cbd3b10f6fa0_ktp_0.jpg

Memproses KTP: /content/drive/MyDrive/ocr_ktp/train/0/IMG-20230729-